If you don't use saturn, install beforehand
- pip install -U transformers accelerate bitsandbytes sentencepiece

In [1]:
import os
#os.environ['HF_HOME'] = '/tmp/cache/'
# os.environ['HF_HOME']

In [2]:
!rm -f minsearch.py

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-03-06 15:06:57--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-03-06 15:06:57 (52.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

## HuggingFace and Google FLAN T5

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [8]:
# let's check how much space we have to download that
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          32G   28G  2.7G  92% /
tmpfs            64M     0   64M   0% /dev
shm              64M     0   64M   0% /dev/shm
/dev/root        29G   13G   17G  44% /vscode
/dev/loop4       32G   28G  2.7G  92% /workspaces
/dev/sdb1        44G   30G   13G  72% /tmp


In [9]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
input_text = "translate English to German: How old are you?" #text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids #turn text into representation

outputs = model.generate(input_ids) #feed representation to the model
result = tokenizer.decode(outputs[0])

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    result = tokenizer.decode(outputs[0])
    return result

In [12]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}
        
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids 
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

**Explanation of Parameters:**

* max_length: Set this to a higher value if you want longer responses. For example, max_length=300.
* num_beams: Increasing this can lead to more thorough exploration of possible sequences. Typical values are between 5 and 10.
* do_sample: Set this to True to use sampling methods. This can produce more diverse responses.
* temperature: Lowering this value makes the model more confident and deterministic, while higher values increase diversity. Typical values range from 0.7 to 1.5.
* top_k and top_p: These parameters control nucleus sampling. top_k limits the sampling pool to the top k tokens, while top_p uses cumulative probability to cut off the sampling pool.

Adjust these based on the desired level of randomness.

In [13]:
rag("I just discovered the course. Can I still join it?")

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'How can we contribute to the course?'